In [15]:
from __future__ import unicode_literals
import classifiers
import re
import nltk
from Politweet import get_tweets, get_transcript
import ratings
from sentiment import polarity_train, classify, prob_classify, plus_df, minus_df
import pandas as pd
pd.set_option('display.max_colwidth', 1200)

In [16]:
tweets = get_tweets("./datasets/tweets.tsv")
transcript = get_transcript("./datasets/transcript.csv")
todo = set(tweets.index)


In [17]:
tweets.to_pickle('./datasets/tweets.pickle')

In [18]:
polarity = polarity_train(tweets)
polarity_minus = set(i for i,_ in classify(minus_df(tweets), polarity))
polarity_plus = set(i for i,_ in classify(plus_df(tweets), polarity))
polarities = tweets.reindex(polarity_minus | polarity_plus).sort()
print len(polarities), "- polarity"
todo -= set(polarities.index)

482 - polarity


In [19]:
others = ratings.all(tweets, ratings.OTHER)
print len(others), "- marked as OTHERS"
todo -= set(others.index)

151 - marked as OTHERS


In [20]:
print len(todo), "- untagged"

2605 - untagged


In [21]:
left = tweets.reindex(todo).sort()

In [22]:
left[["content","tokens", "clean"]][:2]

,content,tokens,clean
tweet.id,,,
936466790,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,"[jim, lehrer, direct, debate, audience, second, tweetdebate]","[{u'lemma': u'jim', u'token': u'jim', u'pos': u'NN'}, {u'lemma': u'lehrer', u'token': u'lehrer', u'pos': u'NN'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936466992,Here we go. #tweetdebate,[tweetdebate],"[{u'lemma': u'go', u'token': u'go', u'pos': u'VBP'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"


In [23]:
for i, tweet in tweets[:3].iterrows():
    print [token for token in tweet["tokens"]]
    
tokens = [" ".join(tweet["tokens"]) for i,tweet in tweets.iterrows()]
tokens[:1]

[u'watch', u'tweetdebate', u'drink', u'wait', u'start', u'cringing', u'mccain', u'blunder']
[u'ahg3', u'michdot', u'yeah', u'slime', u'actually', u'second', u'choice', u'say', u'first', u'one', u'okay', u'roll']
[u'prepare', u'heart', u'attack', u'tweetdebate']


[u'watch tweetdebate drink wait start cringing mccain blunder']

In [24]:
# from collections import Counter
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# count = Counter([token for i, tweet in tweets.iterrows() for token in tweet["tokens"]])

# vectorizer = TfidfVectorizer(lowercase=False)
# tfidf_model = vectorizer.fit_transform(tokens)
# feature_names = vectorizer.get_feature_names()
# df = pd.DataFrame([
#         (feature_names[col], tfidf_model[0, col])
#         for col in tfidf_model.nonzero()[1]])
# df.sort(1, ascending=False)

In [25]:
# for col in tfidf_model.nonzero()[1]:
#     print tfidf_model[0, col]
#     print (feature_names[col], tfidf_model[0, col])

In [26]:
# tweets[tweets["content"].str.contains("zoom")][['content']]

In [41]:
import json
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import sent_tokenize, word_tokenize, FreqDist, WordNetLemmatizer
from nltk.corpus import stopwords
import classifiers

neg = [
    (t['content'], 'neg')
    for i,t in ratings.all(tweets, ratings.NEGATIVE).iterrows()]

pos = [
    (t['content'], 'pos')
    for i,t in ratings.all(tweets, ratings.POSITIVE).iterrows()]

train, test = train_test_split(
    pos + neg, 
    test_size = .2, 
    random_state = 10)


def run_classifier(train, test, vctrzr, clsfr):
    # vectorize with above featurize() function and also vectorize test set with training fit
    X_train = vctrzr.fit_transform([i[0] for i in train])
    X_test = vctrzr.transform([i[0] for i in test])

    # fit the classifier with training data
    clsfr.fit(X_train, [i[1] for i in train])
    
    # grab accuracy
    scr = clsfr.score(X_test, [i[1] for i in test])

    # grab important features
    imp_features = sorted(zip(clsfr.coef_[0], vctrzr.get_feature_names()))
    
    return scr, imp_features

In [55]:
vectorizer = CountVectorizer()
classifier = MultinomialNB()

score, imp_features = run_classifier(train, test, vectorizer, classifier)
score, imp_features[0:20], imp_features[-21:-1]

(0.79347826086956519,
 [(-8.2817239904113915, u"'08"),
  (-8.2817239904113915, u"'no"),
  (-8.2817239904113915, u"'were"),
  (-8.2817239904113915, u'*bored*'),
  (-8.2817239904113915, u'*gasp*'),
  (-8.2817239904113915, u'*holy'),
  (-8.2817239904113915, u'+hesitation'),
  (-8.2817239904113915, u'-1pt'),
  (-8.2817239904113915, u'..http'),
  (-8.2817239904113915, u'..who'),
  (-8.2817239904113915, u'.but'),
  (-8.2817239904113915, u'.tired'),
  (-8.2817239904113915, u'.to'),
  (-8.2817239904113915, u'//bit.ly/2zhw59'),
  (-8.2817239904113915, u'//janosschumacher.wordpress.com/'),
  (-8.2817239904113915, u'//poprl.com/1ln'),
  (-8.2817239904113915, u'//snipr.com/3vlts'),
  (-8.2817239904113915, u'//tinyurl.com/43v3kc'),
  (-8.2817239904113915, u'//tinyurl.com/4dolsf'),
  (-8.2817239904113915, u'//tinyurl.com/5pabty')],
 [(-5.9791388974173456, u'need'),
  (-5.9791388974173456, u'think'),
  (-5.8838287176130208, u'issue'),
  (-5.8838287176130208, u"n't"),
  (-5.8838287176130208, u'nice'),

In [56]:
vectorizer = TfidfVectorizer(tokenizer = featurize)
classifier = LinearSVC()

score, imp_features = run_classifier(train, test, vectorizer, classifier)
score, imp_features[0:20]
score, imp_features[0:20], imp_features[-21:-1]

(0.80434782608695654,
 [(-1.3662136589407419, u"n't"),
  (-1.2755517204200266, u'stop'),
  (-1.1691237214435368, u'yet'),
  (-1.0126262953426366, u'false'),
  (-0.90765961106097681, u'scared'),
  (-0.89515359882356027, u'glazing'),
  (-0.88639306426213349, u'would'),
  (-0.87663353280317768, u'look'),
  (-0.85991625116971737, u'shit'),
  (-0.85385154011999642, u'mccain'),
  (-0.84724695060498179, u'much'),
  (-0.83362521486567753, u'liar'),
  (-0.82612768739431086, u'bracelet'),
  (-0.81656323968661337, u'different'),
  (-0.80828146641973819, u'mention'),
  (-0.78159715475092795, u'economic'),
  (-0.77924594852879059, u'talking'),
  (-0.77757311211144553, u'republican'),
  (-0.76487998275696789, u'threat'),
  (-0.75401452247416489, u'got')],
 [(0.85810726828730854, u'debate'),
  (0.87204986809084051, u'understanding'),
  (0.90998464241723553, u'twitter'),
  (0.92142895089614829, u'festooned'),
  (0.92734672179638789, u'win'),
  (0.96605844647135675, u'clear'),
  (0.97330035409405002, u